<a href="https://colab.research.google.com/github/CarolGitonga/Generative-AI/blob/main/VAEs1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VAE Class defines the encoder and decoder networks, as well as the reparameterization trick. encode() encodes the input into mean and log-variance. reparameterize() samples from the latent space using the mean and log-variance. decode() reconstructs the input from the sampled latent variable. forward() defines the forward pass, encoding the input, sampling from the latent space, and decoding the sample. KL Divergence (KLD): Measures how close the learned latent variable distribution is to the prior distribution.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import binary_cross_entropy

class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim * 2)  # mean and log-variance
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = torch.chunk(h, 2, dim=-1)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def loss_function(recon_x, x, mu, logvar):
   # BCE = binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return KLD

# Hyperparameters
input_dim = 784  # e.g., 28x28 images flattened
hidden_dim = 400
latent_dim = 20
learning_rate = 1e-3

# Model, optimizer, and loss function
model = VAE(input_dim, hidden_dim, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Dummy data for demonstration
inputs = torch.randn(64, input_dim)  # batch size of 64

# Training loop
model.train()
for epoch in range(10):  # number of epochs
    optimizer.zero_grad()
    recon_batch, mu, logvar = model(inputs)
    loss = loss_function(recon_batch, inputs, mu, logvar)
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/10], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 50.1847
Epoch [2/10], Loss: 11.4711
Epoch [3/10], Loss: 7.1784
Epoch [4/10], Loss: 8.3488
Epoch [5/10], Loss: 8.1335
Epoch [6/10], Loss: 7.0724
Epoch [7/10], Loss: 5.9471
Epoch [8/10], Loss: 4.9433
Epoch [9/10], Loss: 4.1102
Epoch [10/10], Loss: 3.4768
